# Import Modules

In [2]:
import h2o
import pandas

In [3]:
project_path = "/gtc-2017"

# Connect or Start H2O

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,43 mins 08 secs
H2O cluster version:,3.11.0.230
H2O cluster version age:,14 hours and 29 minutes
H2O cluster name:,root
H2O cluster total nodes:,1
H2O cluster free memory:,19.02 Gb
H2O cluster total cores:,20
H2O cluster allowed cores:,20
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


# Import Data

Import data into H2O.

In [5]:
mnist_training = h2o.import_file(project_path+"/data/mnist-training.csv")
mnist_testing = h2o.import_file(project_path+"/data/mnist-testing.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Change the "label" column to a factor (i.e. categorical feature).

In [6]:
mnist_training["label"] = mnist_training["label"].asfactor()
mnist_testing["label"] = mnist_testing["label"].asfactor()

# Explore Data

In [7]:
mnist_training.head()

uri,label
/gtc-2017/data/mnist_png/training/6/6453.png,6
/gtc-2017/data/mnist_png/training/6/13970.png,6
/gtc-2017/data/mnist_png/training/6/21967.png,6
/gtc-2017/data/mnist_png/training/6/11277.png,6
/gtc-2017/data/mnist_png/training/6/27350.png,6
/gtc-2017/data/mnist_png/training/6/32119.png,6
/gtc-2017/data/mnist_png/training/6/44201.png,6
/gtc-2017/data/mnist_png/training/6/58770.png,6
/gtc-2017/data/mnist_png/training/6/39440.png,6
/gtc-2017/data/mnist_png/training/6/41988.png,6


# Test GPU Driver

In [8]:
!nvidia-smi

Tue May  9 04:02:34 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 378.13                 Driver Version: 378.13                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:02:00.0      On |                  N/A |
| 27%   35C    P8    11W / 180W |   4756MiB /  8112MiB |     11%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Run Pre-Trained Model

In [11]:
from h2o.estimators.deepwater import H2ODeepWaterEstimator
model_mnist_lenet_mx = H2ODeepWaterEstimator(epochs=80,
                                             network="user",
                                             network_definition_file=project_path+"/models/model_mnist_lenet_mx.json",
                                             network_parameters_file=project_path+"/models/model_mnist_lenet_mx_params",
                                             backend="mxnet",
                                             image_shape=[28,28],
                                             channels=3)
model_mnist_lenet_mx.train(x=["uri"], y="label",
                           training_frame=mnist_training,
                           validation_frame=mnist_testing,
                           model_id="model_mnist_lenet_mx")

deepwater Model Build progress: |█████████████████████████████████████████| 100%


In [12]:
model_mnist_lenet_mx.show()

Model Details
H2ODeepWaterEstimator :  Deep Water
Model Key:  model_mnist_lenet_mx


ModelMetricsMultinomial: deepwater
** Reported on train data. **

MSE: 0.0005918633093176937
RMSE: 0.024328240982810362
LogLoss: 0.0026036111949203113
Mean Per-Class Error: 0.000701177888984224
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
971.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 / 971
0.0,1119.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0008929,"1 / 1,120"
1.0,0.0,956.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0020877,2 / 958
0.0,0.0,1.0,1046.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0009551,"1 / 1,047"
0.0,0.0,0.0,0.0,987.0,0.0,0.0,1.0,0.0,0.0,0.0010121,1 / 988
0.0,0.0,0.0,0.0,0.0,914.0,0.0,0.0,0.0,0.0,0.0,0 / 914
0.0,0.0,0.0,0.0,0.0,0.0,995.0,0.0,0.0,0.0,0.0,0 / 995
0.0,0.0,0.0,0.0,0.0,0.0,0.0,1090.0,0.0,0.0,0.0,"0 / 1,090"
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,987.0,0.0,0.0,0 / 987
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,967.0,0.0020640,2 / 969


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9993027
2,0.9998008
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0
10,1.0



ModelMetricsMultinomial: deepwater
** Reported on validation data. **

MSE: 0.010266314934563295
RMSE: 0.10132282533843642
LogLoss: 0.05577138344284987
Mean Per-Class Error: 0.012468904842945232
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
972.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0081633,8 / 980
0.0,1128.0,2.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0061674,"7 / 1,135"
1.0,0.0,1020.0,2.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0116279,"12 / 1,032"
0.0,0.0,4.0,994.0,0.0,6.0,0.0,4.0,2.0,0.0,0.0158416,"16 / 1,010"
0.0,0.0,0.0,0.0,968.0,0.0,4.0,1.0,0.0,9.0,0.0142566,14 / 982
1.0,0.0,0.0,7.0,1.0,882.0,1.0,0.0,0.0,0.0,0.0112108,10 / 892
4.0,2.0,0.0,1.0,0.0,6.0,944.0,0.0,1.0,0.0,0.0146138,14 / 958
0.0,1.0,5.0,1.0,1.0,0.0,0.0,1019.0,0.0,1.0,0.0087549,"9 / 1,028"
0.0,0.0,0.0,3.0,1.0,2.0,0.0,2.0,964.0,2.0,0.0102669,10 / 974
2.0,2.0,1.0,3.0,7.0,5.0,0.0,3.0,1.0,985.0,0.0237859,"24 / 1,009"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9876
2,0.9983
3,0.9995000
4,0.9998
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2017-05-09 04:08:33,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2017-05-09 04:08:33,0.926 sec,11010 obs/sec,0.0170667,1,1024.0,0.0243282,0.0026036,0.0006973,0.1013228,0.0557714,0.0124
,2017-05-09 04:08:41,8.633 sec,18018 obs/sec,2.2016,129,132096.0,0.0677605,0.0194230,0.0056779,0.1113318,0.0894795,0.0142
,2017-05-09 04:08:46,13.679 sec,18069 obs/sec,3.584,210,215040.0,0.0300980,0.0033284,0.0009961,0.1005264,0.0674123,0.0114
,2017-05-09 04:08:51,18.725 sec,17837 obs/sec,4.8981333,287,293888.0,0.0263203,0.0024401,0.0008965,0.1017703,0.0724684,0.0118
,2017-05-09 04:08:56,23.771 sec,17624 obs/sec,6.1781333,362,370688.0,0.0166112,0.0009901,0.0002988,0.1014168,0.0702716,0.0118
,2017-05-09 04:09:01,28.817 sec,17308 obs/sec,7.3898667,433,443392.0,0.0025335,0.0001035,0.0,0.0981400,0.0664096,0.0109
,2017-05-09 04:09:06,33.871 sec,17064 obs/sec,8.5845333,503,515072.0,0.0014472,0.0000564,0.0,0.0956835,0.0639628,0.0109
,2017-05-09 04:09:11,38.963 sec,16994 obs/sec,9.8474667,577,590848.0,0.0017042,0.0000510,0.0,0.0974253,0.0675619,0.0112
,2017-05-09 04:09:16,44.046 sec,16907 obs/sec,11.0933333,650,665600.0,0.0004598,0.0000244,0.0,0.0962777,0.0675475,0.0105
